# Molecular Interaction Mapping: From ProLIF Analysis to 3D Rendering

This notebook is divided into two parts:

- Detecting interactions in a complex protein-ligand using PROLIF
- Preparing the data for Blender

# Interaction detection with PROLIF

## Required libraries

Was tested with:

- MDAnalysis 2.6.1
- PROLIF 2.0.3

In [ ]:
import MDAnalysis as mda
import nglview as nv # for visualization in the notebook, not necessary
import prolif as plf
import json

## Prepare your Universe

In [5]:
# load topology and trajectory
u = mda.Universe(plf.datafiles.TOP, plf.datafiles.TRAJ)

# create selections for the ligand and protein
ligand_selection = u.select_atoms("resname LIG")
protein_selection = u.select_atoms("protein")
ligand_selection, protein_selection

(<AtomGroup with 79 atoms>, <AtomGroup with 4988 atoms>)

You will need the topology and trajectory files to be loaded in your Blender session once the interaction data is generated.
To get those used in this notebook you can either save your universe into dedicated files or find them in your PROLIF installation.
For the second option they were stored under the following path:

In [ ]:
/home/user/miniforge3/envs/mdanalysis/lib/python3.11/site-packages/prolif/data/
.
├── ...
├── top.pdb
├── traj.xtc
├── ...


In [6]:
w = nv.show_mdanalysis(u)
w

NGLWidget(max_frame=249)

## Fixing issues

Your data might not working working out of the box with PROLIF but thankfully their [tutorials](https://prolif.readthedocs.io/en/stable/source/tutorials.html) provide multiple solutions to make them compatible.
We won't go into these steps as we used the data used in these tutorials which are directly working.

## Detecting the interactions

Be aware that selecting a subgroup of the protein or the ligand will induce an index offset. It will work perfectly for the interaction analysis but it will complicate the preparation of the data (see the section *Convert PROLIF data into a JSON structure*).

In [18]:
# use default interactions
fp = plf.Fingerprint()
# run on a slice of the trajectory frames: from begining to end with a step of 10
fp.run(u.trajectory, ligand_selection, protein_selection)

  0%|          | 0/250 [00:00<?, ?it/s]

<prolif.fingerprint.Fingerprint: 9 interactions: ['Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'] at 0x7fc0e8556950>

# Convert PROLIF data into a JSON structure

When an interaction is detected, PROLIF returns as nested dictionnary that contains the *parent_indices*, which are the indices of the atoms involved in the interaction. For the protein, the values are useable as is but for the ligand, which is generally at the end of the file (always double check !), the indices will be reset. To fix that we need to update our indices.  

In this example we just needed to add the total number of atoms for the protein to the ligand indices. This will depend on your system. Also, if you select a subset of atoms you will need to update the indices accordingly.   

In [ ]:
# Results of the first frame
fp.ifp[0]

{(ResidueId(LIG, 1, G), ResidueId(PHE, 331, B)): {'Hydrophobic': ({'indices': {'ligand': (0,), 'protein': (10,)}, 'parent_indices': {'ligand': (0,), 'protein': (4024,)}, 'distance': 3.8853186637948007},), 'PiStacking': ({'indices': {'ligand': (3, 4, 5, 6, 7, 42), 'protein': (7, 8, 10, 12, 16, 14)}, 'parent_indices': {'ligand': (3, 4, 5, 6, 7, 42), 'protein': (4021, 4022, 4024, 4026, 4030, 4028)}, 'distance': 5.367013639826269, 'plane_angle': 92.60546131577205, 'normal_to_centroid_angle': 25.351669282699163, 'intersect_distance': 1.1815626733259783},), 'VdWContact': ({'indices': {'ligand': (6,), 'protein': (11,)}, 'parent_indices': {'ligand': (6,), 'protein': (4025,)}, 'distance': 2.5662038681336456},)}, (ResidueId(LIG, 1, G), ResidueId(LEU, 126, A)): {'Hydrophobic': ({'indices': {'ligand': (22,), 'protein': (13,)}, 'parent_indices': {'ligand': (22,), 'protein': (1453,)}, 'distance': 4.325043698349231},)}, (ResidueId(LIG, 1, G), ResidueId(CYS, 133, A)): {'Hydrophobic': ({'indices': {'li

In [ ]:
prot_length = len(u.select_atoms("protein"))
prot_length

4988

Without going into too much details you can notice difference treatments for the storage of the different interactions:

- Hydrogen bonds which are connection between two atoms only need two indices, that of the ligand atom and of the protein atom.
- Pi-Stacking which are done between several aromatic atoms must be stored with all the aromatic atoms involved.

The structure might change in next versions but keep in mind that those indices are needed for the representation construction in Blender. Custom changes might break it if you don't know what you do.

In [20]:
dict_inter = {}

for frame, info in fp.ifp.items():
    dict_inter[frame] = {}
    for couple, interactions in info.items():
        for i_type, atoms in list(interactions.items()):
            if i_type not in ["Hydrophobic", "VdWContact"]:

                if i_type in ["HBAcceptor", "HBDonor"]:
                    i_type = "Hydrogen"
                if i_type in ["Cationic", "Anionic"]:
                    i_type = "Salt_Bridge"
                if "Face" in i_type:
                    i_type = "PiStacking"
                if i_type not in dict_inter[frame]:
                    dict_inter[frame][i_type] = []

                parent_indices = atoms[0]['parent_indices']
                lig_indices = [index + prot_length for index in parent_indices['ligand']]
                prot_indices = [index for index in parent_indices['protein']]

                if i_type == "Hydrogen":
                    if len(parent_indices['ligand']) == 1 and len(parent_indices['protein']) == 2:
                        lig_indices = [lig_indices[0]]
                        prot_indices = [prot_indices[1]]
                    if len(parent_indices['ligand']) == 2 and len(parent_indices['protein']) == 1:
                        lig_indices = [lig_indices[1]]

                if i_type == "PiStacking":
                    unique_lig_indices = list(set(lig_indices))
                    unique_prot_indices = list(set(prot_indices))

                    new_entry = {'ligand': unique_lig_indices, 'protein': unique_prot_indices}
                    if new_entry not in dict_inter[frame][i_type]:
                        dict_inter[frame][i_type].append(new_entry)
                else:
                    dict_inter[frame][i_type].append({'ligand': lig_indices, 'protein': prot_indices})

In [ ]:
# First frame interactions
dict_inter[0]

{'Hydrogen': [{'ligand': [5032], 'protein': [2180]},
  {'ligand': [5040], 'protein': [1199]},
  {'ligand': [5016], 'protein': [1988]}],
 'Salt_Bridge': [{'ligand': [5001], 'protein': [1199]}],
 'PiStacking': [{'ligand': [4992, 4993, 4994, 4995, 5030, 4991],
   'protein': [2665, 2666, 2668, 2670, 2672, 2674]}]}

# Save PROLIF data in a JSON file

This JSON file is the only thing needed by the Blender extension to build your interactions. If you use the same interaction types and conserve a coherent structure, you can generate your file without using PROLIF. 

In [ ]:
interaction_data = {}

for frame, interactions in dict_inter.items():
    for interaction_type, interaction_list in interactions.items():
        if interaction_type not in interaction_data:
            interaction_data[interaction_type] = {}

        frame_key = int(frame)
        if frame_key not in interaction_data[interaction_type]:
            interaction_data[interaction_type][frame_key] = []

        for interaction in interaction_list:
            if interaction_type == "PiStacking":
                interaction_data[interaction_type][frame_key].append({
                    "Ligand": interaction["ligand"],
                    "Protein": interaction["protein"]
                })
                continue
            for ligand in interaction['ligand']:
                for protein in interaction['protein']:
                    interaction_data[interaction_type][frame_key].append({
                        "Ligand": round(float(ligand), 5),
                        "Protein": round(float(protein), 5)
                    })

with open("prot_lig_interactions.json", "w") as json_file:
    json.dump(interaction_data, json_file, indent=4)

{
    "0": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
                4022,
                4024,
                4026,
                4028,
                4030
            ]
        }
    ],
    "1": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
                4022,
                4024,
                4026,
                4028,
                4030
            ]
        }
    ],
    "3": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
    

In [ ]:
print(json.dumps(interaction_data['Hydrogen'], indent=4))

{
    "0": [
        {
            "Ligand": 5040.0,
            "Protein": 1489.0
        },
        {
            "Ligand": 5032.0,
            "Protein": 2819.0
        },
        {
            "Ligand": 5016.0,
            "Protein": 2627.0
        }
    ],
    "1": [
        {
            "Ligand": 5040.0,
            "Protein": 1489.0
        },
        {
            "Ligand": 5016.0,
            "Protein": 2627.0
        }
    ],
    "2": [
        {
            "Ligand": 5040.0,
            "Protein": 1490.0
        },
        {
            "Ligand": 5016.0,
            "Protein": 2627.0
        }
    ],
    "3": [
        {
            "Ligand": 5040.0,
            "Protein": 1489.0
        },
        {
            "Ligand": 5016.0,
            "Protein": 2627.0
        }
    ],
    "4": [
        {
            "Ligand": 5040.0,
            "Protein": 1489.0
        },
        {
            "Ligand": 5016.0,
            "Protein": 2627.0
        }
    ],
    "5": [
        {
 

In [29]:
print(json.dumps(interaction_data['PiStacking'], indent=4))

{
    "0": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
                4022,
                4024,
                4026,
                4028,
                4030
            ]
        }
    ],
    "1": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
                4022,
                4024,
                4026,
                4028,
                4030
            ]
        }
    ],
    "3": [
        {
            "Ligand": [
                4992,
                4993,
                4994,
                4995,
                5030,
                4991
            ],
            "Protein": [
                4021,
    

In [ ]:
blender -b protein_ligand.blend -s 0 -e 250 --python ../../../../Blender/Python\ Scripting/Render/MN_sessions.py -a